# Worksheet 08

Name:  Liang Han
UID: U86104920

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [58]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.396329700141304, 6.882948606506545, 6.345221602848693, 6.484345173992915, 5.163827746079461, 4.20621827131756, 5.476825279060595, 5.0280535677518765, 6.394844495149558, 3.0678311580527824]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [59]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.107628101891843, 6.6938792843136685, 7.440575613046905, 7.85027023605155, 7.986151539937686, 8.548294681407482, 7.940192828806045, 7.05723872748297, 7.882844842231778, 8.133638848011401]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [60]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.133638848011401, 7.882844842231778, 3.0678311580527824, 7.05723872748297, 7.940192828806045, 6.394844495149558, 5.0280535677518765, 5.476825279060595, 4.20621827131756, 8.548294681407482]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean and variance, and probability of x coming from specie i (in worksheet, 0.5 since fair coin toss)

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [ ]:
kmeans = KMeans(2, init='k-means++', n_init='auto').fit(X=np.array(data).reshape(-1, 1))

In [87]:
s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1) / len(s1), sum(s2) / len(s2)]
var = [sum(map(lambda x: (x - mean[0]) ** 2, s1)) / len(s1), sum(map(lambda x: (x - mean[1]) ** 2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))
print("")

[8.133638848011401, 7.882844842231778, 7.05723872748297, 7.940192828806045, 6.394844495149558, 8.548294681407482]
[3.0678311580527824, 5.0280535677518765, 5.476825279060595, 4.20621827131756]
[[7.65950907]
 [4.44473207]] 6.463732516826693
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 7.659509070514872,  mean_2 = 4.4447320690457035
var_1 = 0.5175845335691994,  var_2 = 0.839556328852874


mean_1 = 7.659509070514872,  mean_2 = 4.4447320690457035
var_1 = 0.5175845335691994,  var_2 = 0.839556328852874
kmeans seems to have assigned mean_1 to c2, and mean_2 to c1.

The true mean for mean_1 is thus 8, var is 1.
for mean_2 is 5, var is 1.
The variance for mean_1 differs greatly, but the mean is pretty accurate.
The predicted probability is exactly as the true value since we took 4 data from c1 and 6 data from c2 when sampling.



e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [62]:
from scipy.stats import norm

prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []  # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        # prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.133638848011401
probability of observing that point if it came from cluster 0 =  0.5066556681378395
probability of observing that point if it came from cluster 1 =  3.051989045893879e-05
point =  7.882844842231778
probability of observing that point if it came from cluster 0 =  0.7022606753484021
probability of observing that point if it came from cluster 1 =  0.00010845214911427004
point =  3.0678311580527824
probability of observing that point if it came from cluster 0 =  6.269367088414215e-18
probability of observing that point if it came from cluster 1 =  0.12382171115370404
point =  7.05723872748297
probability of observing that point if it came from cluster 0 =  0.39166093163892207
probability of observing that point if it came from cluster 1 =  0.0037514426672387523
point =  7.940192828806045
probability of observing that point if it came from cluster 0 =  0.6653791069021735
probability of observing that point if it came from cluster 1 =  8.179747502410928e-05
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [63]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 7.739256935066205,  mean_2 = 4.606873415547877
var_1 = 0.444073248597588,  var_2 = 1.0631529373051356


mean1 is higher and less variant, while mean2 is higher is more variant compared to the K-means result.
However, the means are very close already.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [64]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []  # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        # prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  8.133638848011401
probability of observing that point if it came from cluster 0 =  0.6056006496930992
probability of observing that point if it came from cluster 1 =  0.0015302595813791425
point =  7.882844842231778
probability of observing that point if it came from cluster 0 =  0.8526142384423634
probability of observing that point if it came from cluster 1 =  0.003254849560784393
point =  3.0678311580527824
probability of observing that point if it came from cluster 0 =  8.394526121909332e-25
probability of observing that point if it came from cluster 1 =  0.13160103172496973
point =  7.05723872748297
probability of observing that point if it came from cluster 0 =  0.2762221200633724
probability of observing that point if it came from cluster 1 =  0.026350932664366732
point =  7.940192828806045
probability of observing that point if it came from cluster 0 =  0.8109543510818454
probability of observing that point if it came from cluster 1 =  0.0027524117493725577
point =  6.

For the 6.394 point, it seems to have changed the assignments. Now the probability of coming from s2 is 0.869, and previously it was 0.354. It seems to have changed the preferred assignment.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [82]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    rand = random.random()
    if rand <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [8.133638848011401, 7.882844842231778, 7.05723872748297, 7.940192828806045, 6.394844495149558, 8.548294681407482]
points in cluster 1 : [3.0678311580527824, 5.0280535677518765, 5.476825279060595, 4.20621827131756]
